<a href="https://colab.research.google.com/github/sebastiaanbv/Proyecto-final/blob/main/Proyeccion_peso_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso 1: Importar librerias necesarias

In [1]:
!pip install catboost
!pip install xgboost
!pip install dcor
!pip install sweetviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 83.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split #Me permite separar el df en test and training.
from sklearn.linear_model import SGDRegressor, SGDClassifier #Se genera los gradientes. SGDRegressor = Rg Lineal y SGDClassifier = Rg Logistica
from sklearn.preprocessing import StandardScaler, OneHotEncoder #Normaliza variables numericas
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_absolute_error, classification_report
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
#from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression,LogisticRegression
import xgboost as xgb
from xgboost import plot_importance, XGBRegressor, XGBClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
#from xgboost import XGBClassifier
#import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
from sklearn import metrics
import psycopg2
import numpy as np
import matplotlib.patches    as mpatches
import sweetviz as sv
from scipy.stats import pearsonr, spearmanr, kendalltau
from sklearn.feature_selection import mutual_info_regression, RFE, SelectKBest, f_classif, chi2, mutual_info_classif
from dcor import distance_correlation
from sklearn.decomposition import PCA
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer


Paso 2: Cargar el dataset

In [3]:
#Leer el dataset
archivo_csv = '/content/datos_granja_ok 1.csv'
df = pd.read_csv(archivo_csv,sep=';')
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/datos_granja_ok 1.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Paso 3: EDA

In [ ]:
df.columns

In [ ]:
df = df.drop(columns=['TipoPollo', 'Ventilacion', 'Genetica', 'Conversion'])
df

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isnull().value_counts()

In [ ]:
print('Recuento de columnas por tipo: ', df.dtypes.value_counts())
print('Sumatoria de valores nulos en el dataframe: ', df.isna().sum())

In [ ]:
# Quitar espacios en blanco en todas las columnas de tipo string
df[['Depto', 'Ciudad','TipoGranja','Sexo']] = df[['Depto', 'Ciudad','TipoGranja','Sexo']].apply(lambda x: x.str.strip())

In [ ]:
df['Temperatura'].value_counts()

In [ ]:
plt.figure(figsize=(12,10))
df['Temperatura'].value_counts()[:15].plot(kind='barh')

In [ ]:
df['Depto'].value_counts()

In [ ]:
plt.figure(figsize=(10,5))
df['Depto'].value_counts()[:15].plot(kind='pie')

In [ ]:
print('la media es: ',df['Consumo'].mean())
print('la mediana es: ',df['Consumo'].median())
print('el valor mínimo de consumo es: : ',df['Consumo'].min())
print('el valor máximo de consumo es: : ',df['Consumo'].max())
print('la desviación éstandar de consumo es: : ',df['Consumo'].std())

In [ ]:
sns.histplot(data = df,x = 'Consumo')
plt.axvline(x=df['Consumo'].mean(),color='red',linestyle='dashed',linewidth=2)

In [ ]:
print('la media de la edad es: ',df['Edad'].mean())
print('la mediana de la edad es: ',df['Edad'].median())
print('la desviación éstandar de la edad es: : ',df['Edad'].std())

In [ ]:
sns.histplot(data = df,x = 'Edad')
plt.axvline(x=df['Edad'].mean(),color='red',linestyle='dashed',linewidth=2)

In [ ]:
df['Sexo'].value_counts()
plt.figure(figsize=(10,5))
df['Sexo'].value_counts()[:15].plot(kind='pie')

In [ ]:
df['TipoGranja'].value_counts()
plt.figure(figsize=(10,5))
df['TipoGranja'].value_counts()[:15].plot(kind='pie')

In [ ]:
print('la media del peso es: ',df['Peso'].mean())
print('la mediana del peso es: ',df['Peso'].median())
print('el valor mínimo del peso  es: : ',df['Peso'].min())
print('el valor máximo del peso  es: : ',df['Peso'].max())
print('la desviación éstandar del peso es: : ',df['Peso'].std())

La función sweet_report = sv.analyze(df) pertenece a la biblioteca Sweetviz de Python, diseñada para la exploración y visualización de datos.
Al ejecutar sv.analyze(df), Sweetviz genera un informe exploratorio completo del DataFrame (df); para ver sus capacidades interactivas se genera en formato HTML.

In [ ]:
sweet_report = sv.analyze(df)
sweet_report.show_html('C:\Alex\EAN ML\Machine Learning\Proyecto Final\Datos\sw_report_datos_granja_ok.csv_.html')

Sacamos correlaciones con diferentes funciones.

In [ ]:
#Estas funciones nos permite ver la correlacion de 2 variables con diferentes funciones, esta fucnjon recibe un datset y 2 dminesiones como parametros y retorna los coeficientes de correlacion entre ellas; explicadas por
#Coeficiente de Corelacion que mide la fuerza y direccion de la relacion lineal. Si es 1 Indica correlacion positiva perfecta, -1 Indica correlacion negativa perfecta y 0 no hay correlacion.
#PValor que indica la significancia estadistica de la correlacion encontrada. Si Pvalor >0.05 la correlacion es significativa.

# Pearson Correlation
def pearson_correlation(df, col1, col2):
    corr, _ = pearsonr(df[col1], df[col2])
    return corr

# Spearman Correlation
def spearman_correlation(df, col1, col2):
    corr, _ = spearmanr(df[col1], df[col2])
    return corr

# Kendall Tau Correlation
def kendall_correlation(df, col1, col2):
    corr, _ = kendalltau(df[col1], df[col2])
    return corr

# Mutual Information (works for numerical features)
def mutual_information(df, target_col, feature_col):
    mi = mutual_info_regression(df[[feature_col]], df[target_col])
    return mi[0]

# Gráfico de Información Mutua entre variables numéricas y el target
def plot_mutual_information(df, target_col):
    numerical_df = filter_numerical_columns(df)
    mi_scores = {col: mutual_information(df, target_col, col) for col in numerical_df.columns if col != target_col}
    plt.figure(figsize=(10, 6))
    plt.bar(mi_scores.keys(), mi_scores.values())
    plt.title(f"Mutual Information with Target: {target_col}")
    plt.xlabel("Variables")
    plt.ylabel("Mutual Information Score")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Pearson Correlation Heatmap for numerical columns
def pearson_correlation_numerical(df,metodo):
    # Filter numerical columns
    numerical_df = filter_numerical_columns(df)

    # Compute Pearson correlation matrix
    corr = numerical_df.corr(method=metodo)

    # Plot the heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, linewidths=0.5)
    plt.title(f"{metodo} Correlation Heatmap (Numerical Variables Only)")
    plt.show()
    return corr

In [ ]:
def plot_scatter_with_regression(df, col1, col2):
    plt.figure(figsize=(8, 6))
    sns.regplot(x=col1, y=col2, data=df, scatter_kws={"s": 50}, line_kws={"color": "red"})
    plt.title(f'Scatter Plot with Regression Line: {col1} vs {col2}')
    plt.xlabel(col1)
    plt.ylabel(col2)
    plt.show()

In [ ]:
# Pearson correlation between x and y
pearson_corr = pearson_correlation(df, 'Peso', 'Consumo')
print(f"Pearson Correlation: {pearson_corr}")

# Spearman correlation between x and y
spearman_corr = spearman_correlation(df, 'Peso', 'Consumo')
print(f"Spearman Correlation: {spearman_corr}")

# Kendall correlation between x and y
kendall_corr = kendall_correlation(df, 'Peso', 'Consumo')
print(f"Kendall Correlation: {kendall_corr}")

# Mutual Information between x and y (numerical only)
mutual_info = mutual_information(df, 'Peso', 'Consumo')
print(f"Mutual Information: {mutual_info}")

# Filter only numerical variables in a DataFrame
def filter_numerical_columns(df):
    # Select only numerical columns (int, float)
    return df.select_dtypes(include=[np.number])

# Plot correlation heatmap
pearson_correlation_numerical(df,"spearman")

# Scatter plot with regression line between x and y
plot_scatter_with_regression(df, 'Edad', 'Peso')

plot_mutual_information(df, 'Peso')

Paso 4: Normalización de datos

In [ ]:
#Este codigo elimina las columnas las cuales consideramos que no aportan al modelo por que no hay datos suficientes o tienen valor 0
df = df.drop(columns=['Fecha','TipoAlimento','Depto','CO'])

In [ ]:
# Codificar las variables independientes
label_encoder = LabelEncoder()

df['Sexo'] = label_encoder.fit_transform(df['Sexo'])
df['Ciudad'] = label_encoder.fit_transform(df['Ciudad'])
df['TipoGranja'] = label_encoder.fit_transform(df['TipoGranja'])



In [ ]:
#En esta funcion queremos estandarizar las variables numericas; sin embargo y por caso de negocio consideramos que no es necesario.

##MM_scaler=MinMaxScaler(feature_range=(-1,1))
SD_scaler=StandardScaler()
##X['Consumo']=MM_scaler.fit_transform(X['Consumo'].values.reshape(-1,1))
##X['Edad']=MM_scaler.fit_transform(X['Edad'].values.reshape(-1,1))
#y=SD_scaler.fit_transform(y.values.reshape(-1,1))
#df['Consumo']=SD_scaler.fit_transform(df['Consumo'].values.reshape(-1,1))
#df['Edad']=SD_scaler.fit_transform(df['Edad'].values.reshape(-1,1))
#df['Unidades']=SD_scaler.fit_transform(df['Unidades'].values.reshape(-1,1))
#df['Peso']=SD_scaler.fit_transform(df['Peso'].values.reshape(-1,1))
df.describe()

REVISIÓN DE DATOS ATÍPICOS

In [ ]:
 ##Crear la figura con subplots: 2 filas, 3 columnas
fig, axes = plt.subplots(2, 3, figsize=(12, 6))  # Ajusta figsize si es necesario

# Primer gráfico: Boxplot de Temperatura
sns.boxplot(ax=axes[0, 0], y=df['Temperatura'])
axes[0, 0].set_title('Boxplot de temperatura')

# Segundo gráfico: Boxplot de Consumo
sns.boxplot(ax=axes[0, 1], y=df['Consumo'])
axes[0, 1].set_title('Boxplot de consumo')

# Tercer gráfico: Boxplot de Unidades
sns.boxplot(ax=axes[0, 2], y=df['Unidades'])
axes[0, 2].set_title('Boxplot de unidades')

# Cuarto gráfico: Boxplot de Edad
sns.boxplot(ax=axes[1, 0], y=df['Edad'])
axes[1, 0].set_title('Boxplot de edad')

# Quinto gráfico: Boxplot de Peso
sns.boxplot(ax=axes[1, 1], y=df['Peso'])
axes[1, 1].set_title('Boxplot de Peso')

# Si hay otro gráfico, por ejemplo, puedes dejar el último espacio vacío o poner un gráfico adicional si es necesario
# axes[1, 2] está vacío en este caso

# Ajustar el layout para que no se solapen los gráficos
plt.tight_layout()

# Mostrar los gráficos
plt.show()


Evidenciamos datos atípicos en todas las variables numéricas, por lo que procedemos a eliminarlos.


Eliminar datos atípicos mediante z-score

In [ ]:
from scipy import stats

columnas = ['Temperatura', 'Consumo', 'Unidades', 'Edad', 'Peso']

# Crear una copia del DataFrame original para almacenar los datos filtrados
df_filtered = df.copy()

# Calcular Z-Score para cada columna y eliminar outliers de manera acumulativa pero correcta
for columna in columnas:
    # Calcular el z-score de cada valor en la columna original
    z_scores = stats.zscore(df_filtered[columna])
    # Filtrar los datos para mantener solo aquellos con un z-score en el rango [-3, 3]
    df_filtered = df_filtered[(z_scores > -3) & (z_scores < 3)]

    # Imprimir cuántos outliers fueron eliminados para esta columna específica
    outliers_count = len(df) - len(df_filtered)
    print(f"Valores atípicos eliminados en la columna {columna}: [{outliers_count} rows]")

# Mostrar el DataFrame resultante sin outliers
print(df_filtered)



In [ ]:
 ##Crear la figura con subplots: 2 filas, 3 columnas
fig, axes = plt.subplots(2, 3, figsize=(12, 6))  # Ajusta figsize si es necesario

# Primer gráfico: Boxplot de Temperatura
sns.boxplot(ax=axes[0, 0], y=df_filtered['Temperatura'])
axes[0, 0].set_title('Boxplot de temperatura')

# Segundo gráfico: Boxplot de Consumo
sns.boxplot(ax=axes[0, 1], y=df_filtered['Consumo'])
axes[0, 1].set_title('Boxplot de consumo')

# Tercer gráfico: Boxplot de Unidades
sns.boxplot(ax=axes[0, 2], y=df_filtered['Unidades'])
axes[0, 2].set_title('Boxplot de unidades')

# Cuarto gráfico: Boxplot de Edad
sns.boxplot(ax=axes[1, 0], y=df_filtered['Edad'])
axes[1, 0].set_title('Boxplot de edad')

# Quinto gráfico: Boxplot de Peso
sns.boxplot(ax=axes[1, 1], y=df_filtered['Peso'])
axes[1, 1].set_title('Boxplot de Peso')

# Si hay otro gráfico, por ejemplo, puedes dejar el último espacio vacío o poner un gráfico adicional si es necesario
# axes[1, 2] está vacío en este caso

# Ajustar el layout para que no se solapen los gráficos
plt.tight_layout()

# Mostrar los gráficos
plt.show()


Una vez los datos normalizados podemos revisar nuevamente las matrices de correlacion mediante diferentes funciones. Pearson, Kendall y Spearman

In [ ]:
# Análisis de correlación que depende del metodo seleccionado en la variable metodo, solo son  aplicables 3 tipos: pearson, spearman y kendall
#Spearman: Adecuada para relaciones no lineales, es robusta frente a outliers
#kendall: Adecuada para relaciones no lineales, es robusta frente a outliers
#Pearson: Mide la relacion lineal entre 2 variables cuantitativas.
metodo = "spearman"
corr_matrix = df_filtered.corr(method=metodo)
plt.figure(figsize=(15,10 ))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title(f'{metodo} Matriz de Correlación')
plt.show()

In [ ]:
# Análisis de correlación que depende del metodo seleccionado en la variable metodo, solo son  aplicables 3 tipos: pearson, spearman y kendall
#Spearman: Adecuada para relaciones no lineales, es robusta frente a outliers
#kendall: Adecuada para relaciones no lineales, es robusta frente a outliers
#Pearson: Mide la relacion lineal entre 2 variables cuantitativas.
metodo = "spearman"
corr_matrix = df_filtered.corr(method=metodo)
plt.figure(figsize=(15,10 ))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title(f'{metodo} Matriz de Correlación')
plt.show()

In [ ]:
#Con esta funcion podemos determinar la fuerza en la relacion de las variables frente a nuestra variable objetivo que es el peso y depende de la funcion de correlacion que seleccionemos
target_corr = corr_matrix["Peso"].sort_values(ascending=False)
print(target_corr)

In [ ]:
# Mean Squared Error (MSE)
def mean_squared_error_metric(df_filtered, col1, col2):
    mse = mean_squared_error(df[col1], df[col2])
    return mse

# Mean Absolute Error (MAE)
def mean_absolute_error_metric(df_filtered, col1, col2):
    mae = mean_absolute_error(df[col1], df[col2])
    return mae

# R-squared (R²)
def r_squared_metric(df_filtered, col1, col2):
    r2 = r2_score(df[col1], df[col2])
    return r2

# Distance Correlation
def distance_correlation_metric(df_filtered, col1, col2):
    dcor = distance_correlation(df[col1], df[col2])
    return dcor

Distance Correlation (correlación de distancia) es una métrica que mide cualquier tipo de dependencia entre dos variables, ya sea lineal o no lineal.
Es útil para detectar relaciones complejas entre variables, ya que no se limita a relaciones lineales como el coeficiente de Pearson.
Interpretación del valor: 0 signifca que no tinen ninguna relacion estadistica y 1 indica dependencia total es decir hay una relación muy fuerte entre las variables.
Es util cuando se sospecha que puede haber una relacion compleja o no lineal entre las variables.

In [ ]:
# Mean Squared Error (MSE) between x and y
mse = mean_squared_error_metric(df_filtered, 'Peso', 'Edad')
print(f"Mean Squared Error: {mse}")

# Mean Absolute Error (MAE) between x and y
mae = mean_absolute_error_metric(df_filtered, 'Peso', 'Edad')
print(f"Mean Absolute Error: {mae}")

# R-squared (R²) between x and y
r2 = r_squared_metric(df_filtered, 'Peso', 'Edad')
print(f"R-squared: {r2}")

# Distance Correlation between x and y
dcor =distance_correlation_metric(df_filtered, 'Peso', 'Consumo')
print(f"Distance Correlation: {dcor}")

Paso 5: Seleccion de variables importantes que aportan al modelo ya sea para prediccion o regresion

La función SelectKBest es una herramienta en la biblioteca Scikit-Learn que selecciona automáticamente las K mejores características de un conjunto de datos, basándose en una métrica de puntuación específica.
Es útil para reducción de dimensionalidad y selección de características, permitiendo mejorar la eficiencia y precisión de los modelos al enfocarse en las variables más relevantes.
Tenemos varias metricas de seleccion o puntuacion de variables: Chi1: para datos categoricos, f_clasiff para analisis de varianza util para clasificación, f_regression Prueba F util para regresion y mutual_info_classif ó mutual_info_regression util para irar dependencias de variables

In [ ]:
# Definir las variables predictoras y la variable objetivo
X = df_filtered.drop(columns=["Peso"])  # "Peso" es la variable objetivo
y = df_filtered["Peso"]

# Seleccionar las k mejores variables (por ejemplo, k=5)
selector = SelectKBest(score_func=mutual_info_regression, k=5)
X_selected = selector.fit_transform(X, y)

# Ver cuáles variables fueron seleccionadas
selected_features = X.columns[selector.get_support()]
print("Las mejores variables:", selected_features)


In [ ]:
# Aplicar Lasso con validación cruzada
lasso = LassoCV(cv=5, random_state=1).fit(X, y)
# Seleccionar las variables con coeficientes distintos de cero
selected_features = X.columns[lasso.coef_ != 0]
print("Las mejores variables seleccionadas por Lasso:", selected_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Evaluar el modelo en el conjunto de prueba
y_pred = lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2=r2_score(y_test, y_pred)
print("Error cuadrático medio en test:", mse)
print("R2 :", r2)

In [ ]:
# Ajustar un modelo de bosque aleatorio
rf = RandomForestRegressor(n_estimators=100, random_state=5)
rf.fit(X, y)

# Ver la importancia de las variables
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances = importances.sort_values(ascending=False)
print("Importancia de las variables:", importances)

# Seleccionar las variables más importantes (por ejemplo, con importancia > 0.1)
selected_features = importances[importances > 0.1].index
print("Las mejores variables seleccionadas por Random Forest:", selected_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Evaluar el modelo en el conjunto de prueba
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2=r2_score(y_test, y_pred)
print("Error cuadrático medio en test:", mse)
print("R2 :", r2)

La función RFE(estimator=modelo_base, n_features_to_select=5) en Scikit-Learn se utiliza para realizar Eliminación Recursiva de Características, un método de selección de características.
Con RFE, el objetivo es seleccionar el subconjunto de variables más relevante para el modelo, mejorando la eficiencia y, en algunos casos, la precisión.
Dentro de los modelos base podemos tener: linearRegression, RandomForestRegressor o SVC

In [ ]:
# Definir el modelo base y aplicar RFE
modelo_base = LinearRegression()
rfe = RFE(estimator=modelo_base, n_features_to_select=5)
X_rfe = rfe.fit_transform(X, y)

# Obtener las variables seleccionadas
selected_features = X.columns[rfe.support_]
print("Las mejores variables seleccionadas por RFE:", selected_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Evaluar el modelo en el conjunto de prueba
y_pred = rfe.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2=r2_score(y_test, y_pred)
print("Error cuadrático medio en test:", mse)
print("R2 :", r2)

In [ ]:
# Suponiendo que `X` contiene las variables predictoras y `y` la variable objetivo
# Define y entrena el modelo de XGBoost
modelo_xgb = xgb.XGBRegressor(objective="reg:squarederror", random_state=1)
modelo_xgb.fit(X, y)

# Obtener la importancia de cada variable
importancia = modelo_xgb.feature_importances_

# Convertir la importancia en un DataFrame para ordenar y visualizar
importancia_df = pd.DataFrame({
    'Variable': X.columns,
    'Importancia': importancia
}).sort_values(by='Importancia', ascending=False)

print("Importancia de las variables:")
print(importancia_df)

# Graficar la importancia de variables
plt.figure(figsize=(10, 6))
plot_importance(modelo_xgb, max_num_features=10)  # Puedes ajustar el número de variables a mostrar
plt.title("Importancia de las Variables según XGBoost")
plt.show()

pred = modelo_xgb.predict(X_test)

print("R^2:",metrics.r2_score(y_test, pred))
print("Adjusted R^2:",1 - (1-metrics.r2_score(y_test, pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print("MAE:",metrics.mean_absolute_error(y_test, pred))
print("MSE:",metrics.mean_squared_error(y_test, pred))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, pred)))

In [ ]:
top_k = 5
mejores_variables = importancia_df.head(top_k)['Variable'].values
print("Las mejores variables seleccionadas por XGBoost:", mejores_variables)

El VIF (Variance Inflation Factor o Factor de Inflación de Varianza) es una métrica usada en machine learning y estadística para medir la multicolinealidad entre las variables predictoras de un modelo, especialmente en regresión lineal. La multicolinealidad ocurre cuando dos o más variables independientes están altamente correlacionadas, lo que puede hacer que los coeficientes de un modelo sean inestables y difíciles de interpretar.

Interpretación de los valores de VIF
VIF = 1: La variable no tiene correlación con otras variables, lo que es ideal.
1 < VIF < 5: Existe una correlación moderada con otras variables, lo que generalmente es aceptable.
VIF > 5: La multicolinealidad es significativa; debes investigar si puedes reducir la correlación.
VIF > 10: Muy alta multicolinealidad. Generalmente, es una señal de que una o más variables deben eliminarse o transformarse para reducir la correlación.
El VIF es, por tanto, una herramienta importante en la selección de características para crear modelos más estables y eficientes en machine learning, especialmente en modelos lineales o donde se requiere una interpretación clara de los coeficientes.

In [ ]:
# Supongamos que 'df' es tu DataFrame con variables independientes
#X = df[['Edad','Humedad','consumoAgua','Temperatura','PesoGr']]  # Selecciona tus variables independientes
#X = df[['Temperatura','Consumo','PesoGr','Unidades','Edad','Sexo', 'TipoGranja','Humedad','HorasLuz','CalidadCama','consumoAgua']]  # Selecciona tus variables independientes
# Calcula el VIF para cada variable
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

In [ ]:
#PCA para la importancia de características con 3 componentes principales

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pca = PCA(n_components=3)  # Limitar a 3 componentes principales #USANDO 3 COMPONENTES PRINCIPALES.
X_pca = pca.fit_transform(X_train)

# Importancia de características por componente principal
pca_components = pd.DataFrame(pca.components_, columns=X_train.columns)
pca_components.index = [f'Componente {i+1}' for i in range(len(pca_components))]

# Gráfico de Importancia de Características por Componente Principal
plt.figure(figsize=(12, 8))
sns.heatmap(pca_components, cmap='coolwarm', fmt=".2f")
plt.title('Importancia de Características por Componente Principal (PCA)')
plt.show()

# Visualizar contribuciones de las características a los tres componentes principales
plt.figure(figsize=(12, 8))
pca_components.T.plot(kind='bar', figsize=(10, 8))
plt.title('Contribuciones de Características a los Tres Primeros Componentes Principales')
plt.xlabel('Características')
plt.ylabel('Contribución')
plt.xticks(rotation=90)
plt.show()

Paso 6: Seleccion de las mejores variables para prediccion y modelo

In [ ]:
X = df_filtered[['Temperatura', 'Edad', 'Sexo', 'TipoGranja', 'Consumo']] # "Peso" es la variable objetivo
y = df_filtered["Peso"]
X

In [ ]:

from sklearn.model_selection import RandomizedSearchCV


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Espacio de hiperparámetros para RandomForest
rf_param_grid = {
    'n_estimators': np.arange(50, 200, 10),
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

# Espacio de hiperparámetros para XGBoost
xgb_param_grid = {
    'n_estimators': np.arange(50, 200, 10),
    'max_depth': np.arange(3, 10),
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

# Inicializar modelos
rf_model = RandomForestRegressor()
xgb_model = XGBRegressor(objective='reg:squarederror')

# Configuración de RandomizedSearchCV para RandomForest
rf_random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=rf_param_grid,
    n_iter=50,
    scoring='r2',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Configuración de RandomizedSearchCV para XGBoost
xgb_random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xgb_param_grid,
    n_iter=50,
    scoring='r2',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Ajustar el modelo RandomForest con RandomizedSearchCV
rf_random_search.fit(X_train, y_train)
print("Mejores parámetros para RandomForest:", rf_random_search.best_params_)
print("Mejor puntuación R² para RandomForest:", rf_random_search.best_score_)

# Ajustar el modelo XGBoost con RandomizedSearchCV
xgb_random_search.fit(X_train, y_train)
print("Mejores parámetros para XGBoost:", xgb_random_search.best_params_)
print("Mejor puntuación R² para XGBoost:", xgb_random_search.best_score_)


In [ ]:

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Lista de modelos a evaluar
# Definir los mejores parámetros para RandomForest
best_rf_params = {
    'n_estimators': 100,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'sqrt',
    'max_depth': None,
    'bootstrap': False
}

# Definir los mejores parámetros para XGBoost
best_xgb_params = {
    'subsample': 0.8,
    'reg_lambda': 2,
    'reg_alpha': 0,
    'n_estimators': 180,
    'max_depth': 8,
    'learning_rate': 0.1,
    'gamma': 0,
    'colsample_bytree': 0.8
}



# Inicializar modelos con los mejores parámetros ajustados para evitar overfitting
rf_model_best = RandomForestRegressor(**best_rf_params)
xgb_model_best = XGBRegressor(objective='reg:squarederror', **best_xgb_params)

# Ajustar el modelo RandomForest a los datos normalizados
rf_model_best.fit(X_train, y_train)
print("Modelo RandomForest ajustado con los mejores parámetros regularizados.")

# Ajustar el modelo XGBoost a los datos normalizados
xgb_model_best.fit(X_train, y_train)
print("Modelo XGBoost ajustado con los mejores parámetros regularizados.")

# Evaluar los modelos en el conjunto de prueba
models = {'RandomForest': rf_model_best, 'XGBoost': xgb_model_best}

# Mostrar las métricas de entrenamiento y prueba para cada modelo
for name, model in models.items():
    # Predecir en el conjunto de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calcular métricas
    train_MAE = mean_absolute_error(y_train, y_train_pred)
    test_MAE = mean_absolute_error(y_test, y_test_pred)
    train_MSE = mean_squared_error(y_train, y_train_pred)
    test_MSE = mean_squared_error(y_test, y_test_pred)
    train_R2 = r2_score(y_train, y_train_pred)
    test_R2 = r2_score(y_test, y_test_pred)

    # Imprimir las métricas
    print(f"\n{name} Model Metrics:")
    print(f"  Train MAE: {train_MAE:.4f}")
    print(f"  Test MAE: {test_MAE:.4f}")
    print(f"  Train MSE: {train_MSE:.4f}")
    print(f"  Test MSE: {test_MSE:.4f}")
    print(f"  Train R2: {train_R2:.4f}")
    print(f"  Test R2: {test_R2:.4f}")


In [ ]:
# Lista de modelos a evaluar
models = {

    'Random Forest': RandomForestRegressor(**best_rf_params),
    'XGBoost': XGBRegressor(objective='reg:squarederror', **best_xgb_params),
}

# Configuración de KFold
kf = KFold(n_splits=7, shuffle=True, random_state=42)

# Función para calcular R² ajustado
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Métricas personalizadas
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score),
}

# Número de muestras y características
n_samples, n_features = X.shape

# Evaluación de modelos
for name, model in models.items():
    print(f"\nEvaluando {name}")

    results = cross_validate(
        model,
        X,
        y,
        cv=kf,
        scoring=scoring,
        return_train_score=True
    )

    # Cálculo de R² ajustado para cada división
    train_r2_adj = [adjusted_r2(r2, n_samples, n_features) for r2 in results['train_R2']]
    test_r2_adj = [adjusted_r2(r2, n_samples, n_features) for r2 in results['test_R2']]

    # Mostrar resultados promedio
    print("Métricas de prueba")
    print(f"MAE: {np.mean(results['test_MAE'])}")
    print(f"MSE: {np.mean(results['test_MSE'])}")
    print(f"R2: {np.mean(results['test_R2'])}")
    print(f"R2 Ajustado: {np.mean(test_r2_adj)}")

    print("\nMétricas de entrenamiento")
    print(f"MAE: {np.mean(results['train_MAE'])}")
    print(f"MSE: {np.mean(results['train_MSE'])}")
    print(f"R2: {np.mean(results['train_R2'])}")
    print(f"R2 Ajustado: {np.mean(train_r2_adj)}")

In [ ]:
# Graficar las métricas de entrenamiento y prueba en la misma gráfica para cada partición
fig, axes = plt.subplots(4, 1, figsize=(10, 16), sharex=True)
fig.suptitle("Métricas de Entrenamiento y Prueba por Partición")

# Graficar MAE
axes[0].plot(results['train_MAE'], marker='o', linestyle='-', label='Train MAE')
axes[0].plot(results['test_MAE'], marker='o', linestyle='--', label='Test MAE')
axes[0].set_ylabel("MAE")
axes[0].legend()
axes[0].grid(True)

# Graficar MSE
axes[1].plot(results['train_MSE'], marker='o', linestyle='-', label='Train MSE')
axes[1].plot(results['test_MSE'], marker='o', linestyle='--', label='Test MSE')
axes[1].set_ylabel("MSE")
axes[1].legend()
axes[1].grid(True)

# Graficar R2
axes[2].plot(results['train_R2'], marker='o', linestyle='-', label='Train R2')
axes[2].plot(results['test_R2'], marker='o', linestyle='--', label='Test R2')
axes[2].set_ylabel("R2")
axes[2].legend()
axes[2].grid(True)

# Graficar R2 ajustado
axes[3].plot(train_r2_adj, marker='o', linestyle='-', label='Train R2 Ajustado')
axes[3].plot(test_r2_adj, marker='o', linestyle='--', label='Test R2 Ajustado')
axes[3].set_ylabel("R2 Ajustado")
axes[3].legend()
axes[3].grid(True)

axes[-1].set_xlabel("Partición")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
#Temperatura',  'Edad', 'Sexo', 'TipoGranja','Consumo'
# Nuevos datos para predicción
print(X)


  # Nuevos datos para predicción
nuevos_datos = pd.DataFrame({

    'Temperatura':[31],
    'Edad':[31],
    'Sexo':[1],
    'TipoGranja':[1],
    'Consumo':[586]
})

# Realizar predicciones
prediccion_peso = xgb_model_best.predict(nuevos_datos)
print("El peso seria de:", prediccion_peso[0])